[View in Colaboratory](https://colab.research.google.com/github/Curiousss/InkerIntern/blob/master/FER_WINO_SEPARABLE_NO_CUDNN.ipynb)

In [1]:
!wget https://github.com/LamUong/FacialExpressionRecognition/blob/master/badtrainingdata.txt

--2018-06-22 10:35:21--  https://github.com/LamUong/FacialExpressionRecognition/blob/master/badtrainingdata.txt
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘badtrainingdata.txt’

badtrainingdata.txt     [ <=>                ]  61.91K   378KB/s    in 0.2s    

2018-06-22 10:35:21 (378 KB/s) - ‘badtrainingdata.txt’ saved [63395]



In [3]:
from google.colab import files
uploaded = files.upload()

Saving fer2013.tar to fer2013.tar


In [3]:
!ls

datalab  fer2013.tar  fer2013.tar.gz


In [42]:
import os
!export TF_ENABLE_WINOGRAD_NONFUSED=1
os.environ['TF_ENABLE_WINOGRAD_NONFUSED'] = '1'
os.environ.pop('TF_ENABLE_WINOGRAD_NONFUSED', None)

'1'

In [4]:
!tar xvf fer2013.tar
!ls

fer2013/fer2013.csv
fer2013/README
fer2013/fer2013.bib
fer2013/
badtrainingdata.txt  datalab  fer2013  fer2013.tar


In [5]:

import csv
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D, InputLayer
from keras.layers import Convolution2D, SeparableConv2D, MaxPooling2D, BatchNormalization 
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
img_rows, img_cols = 48, 48
batch_size = 128
classes = 7
epoch = 100
img_channels = 1

In [0]:
import csv
f = open('fer2013/fer2013.csv')
csv_f = csv.reader(f)


In [0]:
train_x = []
train_y = []
val_x =[]
val_y =[]

In [0]:
num=0
for row in csv_f:
  #print(row)
  num = num +1
  #print(num)
  temp_list = []
  for pixel in row[1].split( ):
    temp_list.append(int(pixel))

  if str(row[2]) == "Training":
    train_y.append(int(row[0]))
    train_x.append(temp_list) 
  elif str(row[2]) == "PublicTest":
    val_y.append(int(row[0]))
    val_x.append(temp_list)

In [0]:
train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
val_x = np.asarray(val_x)
val_y = np.asarray(val_y)

In [0]:
train_x = train_x.reshape(train_x.shape[0], 48, 48)
train_x = train_x.reshape(train_x.shape[0], 48, 48, 1 )
train_y = np_utils.to_categorical(train_y, 7)

In [0]:
val_x = val_x.reshape(val_x.shape[0], 48, 48)
val_x = val_x.reshape(val_x.shape[0], 48, 48, 1)
val_y = np_utils.to_categorical(val_y, 7)

In [15]:
#print(train_x[1].shape)
#train_x_pil = train_x.reshape(train_x.shape[0], 48, 48)
#from PIL import Image
#from numpy import array
#img = Image.fromarray(train_x[1])
#img.save("output.png")
print(train_x.shape)

(28709, 48, 48, 1)


In [0]:
train_x = train_x.astype('float32')
train_x = train_x / 255.0
val_x = val_x.astype('float32')
val_x = val_x / 255.0
train_x = train_x - 0.5
train_x = train_x * 2
val_x = val_x - 0.5
val_x = val_x * 2


In [0]:
input_shape = (img_rows, img_cols, img_channels)
model = Sequential()
model.add(SeparableConv2D(filters=64, kernel_size=(3, 3), padding='same',
                            name='image_array', input_shape=input_shape))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(SeparableConv2D(filters=64, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.25))

model.add(SeparableConv2D(filters=128, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(SeparableConv2D(filters=128, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.25))

model.add(SeparableConv2D(filters=256, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(SeparableConv2D(filters=256, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.25))

model.add(SeparableConv2D(filters=512, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(SeparableConv2D(filters=512, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(.25))

#model.add(Flatten())
'''
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(.5))

model.add(Dense(2048))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dropout(.5))
'''
model.add(InputLayer(input_shape=(3, 3, 1024)))
model.add(GlobalAveragePooling2D())
model.add(Dense(7))
model.add(Activation('softmax'))

In [51]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_array (SeparableConv2D (None, 48, 48, 64)        137       
_________________________________________________________________
batch_normalization_27 (Batc (None, 48, 48, 64)        256       
_________________________________________________________________
leaky_re_lu_27 (LeakyReLU)   (None, 48, 48, 64)        0         
_________________________________________________________________
separable_conv2d_22 (Separab (None, 48, 48, 64)        4736      
_________________________________________________________________
batch_normalization_28 (Batc (None, 48, 48, 64)        256       
_________________________________________________________________
leaky_re_lu_28 (LeakyReLU)   (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 24, 24, 64)        0         
__________

In [0]:
model.compile(optimizer='Adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
filepath='Model.best.hdf5'
checkpointer = keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')


In [0]:
model.load_weights('Model.best.hdf5')

In [22]:
# LReLU
import time
start_time = time.time()

model.fit(train_x, train_y, epochs=50, batch_size=batch_size, validation_data=(val_x, val_y), callbacks=[checkpointer])
print("--- %s seconds ---" % (time.time() - start_time))

Train on 28709 samples, validate on 3589 samples
Epoch 1/50
28709/28709 [==============================] - 53s 2ms/step - loss: 0.4090 - acc: 0.8515 - val_loss: 1.2423 - val_acc: 0.6330

Epoch 00001: val_acc did not improve from 0.63918
Epoch 2/50
25856/28709 [==========================>...] - ETA: 5s - loss: 0.4142 - acc: 0.8503

28709/28709 [==============================] - 53s 2ms/step - loss: 0.4188 - acc: 0.8481 - val_loss: 1.2831 - val_acc: 0.6325

Epoch 00002: val_acc did not improve from 0.63918
Epoch 3/50
28709/28709 [==============================] - 53s 2ms/step - loss: 0.4061 - acc: 0.8503 - val_loss: 1.3161 - val_acc: 0.6364

Epoch 00003: val_acc did not improve from 0.63918
Epoch 4/50
 9984/28709 [=========>....................] - ETA: 33s - loss: 0.3785 - acc: 0.8617

28709/28709 [==============================] - 53s 2ms/step - loss: 0.3902 - acc: 0.8598 - val_loss: 1.2826 - val_acc: 0.6459

Epoch 00004: val_acc improved from 0.63918 to 0.64586, saving model to Model.best.hdf5
Epoch 5/50
28709/28709 [==============================] - 53s 2ms/step - loss: 0.3885 - acc: 0.8585 - val_loss: 1.3070 - val_acc: 0.6400

Epoch 00005: val_acc did not improve from 0.64586
Epoch 6/50
 4480/28709 [===>..........................] - ETA: 43s - loss: 0.3332 - acc: 0.8835

28709/28709 [==============================] - 53s 2ms/step - loss: 0.3873 - acc: 0.8579 - val_loss: 1.2978 - val_acc: 0.6428

Epoch 00006: val_acc did not improve from 0.64586
Epoch 7/50
28709/28709 [==============================] - 53s 2ms/step - loss: 0.3801 - acc: 0.8630 - val_loss: 1.3513 - val_acc: 0.6233

Epoch 00007: val_acc did not improve from 0.64586
Epoch 8/50
 6144/28709 [=====>........................] - ETA: 40s - loss: 0.3556 - acc: 0.8722

28709/28709 [==============================] - 54s 2ms/step - loss: 0.3744 - acc: 0.8636 - val_loss: 1.2889 - val_acc: 0.6261

Epoch 00008: val_acc did not improve from 0.64586
Epoch 9/50
28709/28709 [==============================] - 53s 2ms/step - loss: 0.3630 - acc: 0.8674 - val_loss: 1.3062 - val_acc: 0.6286

Epoch 00009: val_acc did not improve from 0.64586
Epoch 10/50
 6400/28709 [=====>........................] - ETA: 39s - loss: 0.3345 - acc: 0.8784

...

28709/28709 [==============================] - 53s 2ms/step - loss: 0.2501 - acc: 0.9110 - val_loss: 1.4443 - val_acc: 0.6358

Epoch 00042: val_acc did not improve from 0.64586
Epoch 43/50
28709/28709 [==============================] - 53s 2ms/step - loss: 0.2474 - acc: 0.9130 - val_loss: 1.5598 - val_acc: 0.6291

Epoch 00043: val_acc did not improve from 0.64586
Epoch 44/50
 6400/28709 [=====>........................] - ETA: 39s - loss: 0.2251 - acc: 0.9223

28709/28709 [==============================] - 53s 2ms/step - loss: 0.2428 - acc: 0.9138 - val_loss: 1.4691 - val_acc: 0.6339

Epoch 00044: val_acc did not improve from 0.64586
Epoch 45/50
28709/28709 [==============================] - 53s 2ms/step - loss: 0.2435 - acc: 0.9140 - val_loss: 1.5082 - val_acc: 0.6378

Epoch 00045: val_acc did not improve from 0.64586
Epoch 46/50
 6400/28709 [=====>........................] - ETA: 39s - loss: 0.2304 - acc: 0.9177

28709/28709 [==============================] - 53s 2ms/step - loss: 0.2380 - acc: 0.9154 - val_loss: 1.4654 - val_acc: 0.6411

Epoch 00046: val_acc did not improve from 0.64586
Epoch 47/50
28709/28709 [==============================] - 53s 2ms/step - loss: 0.2344 - acc: 0.9168 - val_loss: 1.4999 - val_acc: 0.6406

Epoch 00047: val_acc did not improve from 0.64586
Epoch 48/50
 6400/28709 [=====>........................] - ETA: 39s - loss: 0.2232 - acc: 0.9225

28709/28709 [==============================] - 53s 2ms/step - loss: 0.2366 - acc: 0.9157 - val_loss: 1.4765 - val_acc: 0.6453

Epoch 00048: val_acc did not improve from 0.64586
Epoch 49/50
28709/28709 [==============================] - 53s 2ms/step - loss: 0.2402 - acc: 0.9148 - val_loss: 1.5056 - val_acc: 0.6417

Epoch 00049: val_acc did not improve from 0.64586
Epoch 50/50
 6400/28709 [=====>........................] - ETA: 39s - loss: 0.2144 - acc: 0.9259

28709/28709 [==============================] - 53s 2ms/step - loss: 0.2343 - acc: 0.9174 - val_loss: 1.5180 - val_acc: 0.6333

Epoch 00050: val_acc did not improve from 0.64586
--- 2664.325827598572 seconds ---


In [50]:
# input normalized
import time
start_time = time.time()

model.fit(train_x, train_y, epochs=10, batch_size=batch_size, validation_data=(val_x, val_y))
print("--- %s seconds ---" % (time.time() - start_time))

Train on 28709 samples, validate on 3589 samples
Epoch 1/10
28709/28709 [==============================] - 162s 6ms/step - loss: 2.0189 - acc: 0.2407 - val_loss: 1.8390 - val_acc: 0.2856
Epoch 2/10
28709/28709 [==============================] - 156s 5ms/step - loss: 1.8329 - acc: 0.2909 - val_loss: 1.7546 - val_acc: 0.3391
Epoch 3/10
  896/28709 [..............................] - ETA: 2:27 - loss: 1.7662 - acc: 0.3058

28709/28709 [==============================] - 157s 5ms/step - loss: 1.7695 - acc: 0.3230 - val_loss: 1.8986 - val_acc: 0.3093
Epoch 4/10
28709/28709 [==============================] - 157s 5ms/step - loss: 1.7122 - acc: 0.3484 - val_loss: 1.6657 - val_acc: 0.3611
Epoch 5/10
10880/28709 [==========>...................] - ETA: 1:34 - loss: 1.6901 - acc: 0.3509

28709/28709 [==============================] - 156s 5ms/step - loss: 1.6773 - acc: 0.3604 - val_loss: 1.7014 - val_acc: 0.3494
Epoch 6/10
28709/28709 [==============================] - 156s 5ms/step - loss: 1.6553 - acc: 0.3702 - val_loss: 1.6194 - val_acc: 0.3859
Epoch 7/10
13312/28709 [============>.................] - ETA: 1:20 - loss: 1.6141 - acc: 0.3857

28709/28709 [==============================] - 156s 5ms/step - loss: 1.6129 - acc: 0.3857 - val_loss: 1.6001 - val_acc: 0.3923
Epoch 8/10
28709/28709 [==============================] - 156s 5ms/step - loss: 1.5920 - acc: 0.3924 - val_loss: 1.6594 - val_acc: 0.4035
Epoch 9/10
13824/28709 [=============>................] - ETA: 1:18 - loss: 1.5641 - acc: 0.4065

28709/28709 [==============================] - 157s 5ms/step - loss: 1.5559 - acc: 0.4044 - val_loss: 1.5507 - val_acc: 0.4113
Epoch 10/10
28709/28709 [==============================] - 157s 5ms/step - loss: 1.5341 - acc: 0.4100 - val_loss: 1.5732 - val_acc: 0.4068
--- 1574.3259291648865 seconds ---


In [0]:

import time
start_time = time.time()

datagen = ImageDataGenerator(
    featurewise_center=True,  # set input mean to 0 over the dataset
    samplewise_center=True,  # set each sample mean to 0
    featurewise_std_normalization=True,  # divide inputs by std of the dataset
    samplewise_std_normalization=True,  # divide each input by its std
    zca_whitening=True,  # apply ZCA whitening
    rotation_range=40,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True)  # randomly flip images

datagen.fit(train_x)

model.fit_generator(datagen.flow(train_x, train_y,
                    batch_size=batch_size),
                    steps_per_epoch=(train_x.shape[0]/batch_size),
                    epochs=10,
                    validation_data=(val_x, val_y),
                    callbacks=[checkpointer])
print("--- %s seconds ---" % (time.time() - start_time))